In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import json
import random
import scipy
import sklearn
import string
from sklearn import linear_model

In [2]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [3]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
connect_4 = fetch_ucirepo(id=26) 
  
# data (as pandas dataframes) 
X = connect_4.data.features 
y = connect_4.data.targets 
  
# metadata 
print(connect_4.metadata) 
  
# variable information 
print(connect_4.variables) 

{'uci_id': 26, 'name': 'Connect-4', 'repository_url': 'https://archive.ics.uci.edu/dataset/26/connect+4', 'data_url': 'https://archive.ics.uci.edu/static/public/26/data.csv', 'abstract': 'Contains connect-4 positions', 'area': 'Games', 'tasks': ['Classification'], 'characteristics': ['Multivariate', 'Spatial'], 'num_instances': 67557, 'num_features': 42, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1995, 'last_updated': 'Sat Mar 09 2024', 'dataset_doi': '10.24432/C59P43', 'creators': ['John Tromp'], 'intro_paper': None, 'additional_info': {'summary': 'This database contains all legal 8-ply positions in the game of connect-4 in which neither player has won yet, and in which the next move is not forced.\r\n\r\nx is the first player; o the second.\r\n\r\nThe outcome class is the game theoretical value for the first player.', 'purpose': None, 'funded_b

In [97]:
X

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6
0,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
1,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
2,b,b,b,b,b,b,o,b,b,b,...,b,b,b,b,b,b,b,b,b,b
3,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
4,o,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67552,x,x,b,b,b,b,o,x,o,b,...,b,b,b,b,o,o,x,b,b,b
67553,x,x,b,b,b,b,o,b,b,b,...,b,b,b,b,o,x,o,o,x,b
67554,x,x,b,b,b,b,o,o,b,b,...,b,b,b,b,o,x,x,o,b,b
67555,x,o,b,b,b,b,o,b,b,b,...,b,b,b,b,o,x,o,x,x,b


In [98]:
connect4_data = X
connect4_data['outcome'] = y['class']

In [99]:
connect4_data = connect4_data[connect4_data['outcome'] != 'draw']

In [100]:
connect4_data

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f4,f5,f6,g1,g2,g3,g4,g5,g6,outcome
0,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
1,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
2,b,b,b,b,b,b,o,b,b,b,...,b,b,b,b,b,b,b,b,b,win
3,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
4,o,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67549,x,x,o,x,b,b,o,b,b,b,...,b,b,b,o,o,x,b,b,b,loss
67550,x,b,b,b,b,b,o,o,x,o,...,b,b,b,x,b,b,b,b,b,loss
67551,x,x,o,b,b,b,o,o,x,b,...,b,b,b,x,o,b,b,b,b,loss
67552,x,x,b,b,b,b,o,x,o,b,...,b,b,b,o,o,x,b,b,b,loss


In [101]:
grid = [['a1', 'b1', 'c1', 'd1', 'e1', 'f1', 'g1'],
 ['a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2'],
 ['a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3'],
 ['a4', 'b4', 'c4', 'd4', 'e4', 'f4', 'g4'],
 ['a5', 'b5', 'c5', 'd5', 'e5', 'f5', 'g5'],
 ['a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6']]

In [102]:
def center_column_pieces(board, grid, chip):
    rows = len(grid)
    col = 3
    count_x = 0
    count_o = 0
    
    # Check columns for three in a row
    for row in range(rows):
        if board[grid[row][3]] == 'b':
            break
        if board[grid[row][3]] == chip:
            count_x += 1
        else:
            count_o += 1
            
    return count_x, count_o

In [103]:
def bottom_row_pieces(board, grid, chip):
    rows = 0
    col = len(grid[0])
    count_x = 0
    count_o = 0
    
    # Check columns for three in a row
    for c in range(col):
        if board[grid[0][c]] == 'b':
            continue
        if board[grid[0][c]] == chip:
            count_x += 1
        else:
            count_o += 1
            
    return count_x, count_o

def detect_three_in_a_row(board, grid, chip):
    rows = len(grid)
    cols = len(grid[0])
    
    # Check rows for three in a row
    for row in range(rows):
        for col in range(cols - 2):
            if board[grid[row][col]] == board[grid[row][col + 1]] == board[grid[row][col + 2]] == chip:
                return True

    # Check columns for three in a row
    for col in range(cols):
        for row in range(rows - 2):
            if board[grid[row][col]] == 'b':
                break
            if board[grid[row][col]] == board[grid[row + 1][col]] == board[grid[row + 2][col]] == chip:
                return True
    return False
"""
    # Check diagonals (top-left to bottom-right)
    for row in range(rows - 2):
        for col in range(cols - 2):
            if board[grid[row][col]] == board[grid[row + 1][col + 1]] == board[grid[row + 2][col + 2]] == chip:
                return True

    # Check diagonals (bottom-left to top-right)
    for row in range(2, rows):
        for col in range(cols - 2):
            if board[grid[row][col]] == board[grid[row - 1][col + 1]] == board[grid[row - 2][col + 2]] == chip:
                return True
"""

In [104]:
for index, row in connect4_data.iterrows():
    row_dict = row.to_dict()
    player1_centercol, player2_centercol = center_column_pieces(row_dict, grid, 'x')
    player1_bottomrow, player2_bottomrow = bottom_row_pieces(row_dict, grid, 'x')
    
    connect4_data.at[index, 'player1_centercol'] = player1_centercol
    connect4_data.at[index, 'player1_bottomrow'] = player1_bottomrow
    connect4_data.at[index, 'player2_centercol'] = player2_centercol
    connect4_data.at[index, 'player2_bottomrow'] = player2_bottomrow

/var/folders/l4/4q8rhj6d5r1dg7py8kp2xv0w0000gn/T/ipykernel_906/1034605639.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connect4_data.at[index, 'player1_centercol'] = player1_centercol
/var/folders/l4/4q8rhj6d5r1dg7py8kp2xv0w0000gn/T/ipykernel_906/1034605639.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connect4_data.at[index, 'player1_bottomrow'] = player1_bottomrow
/var/folders/l4/4q8rhj6d5r1dg7py8kp2xv0w0000gn/T/ipykernel_906/1034605639.py:8: SettingWithCopyWarning: 
A value is trying to be s

In [105]:
# Separate the feature columns and the outcome column
features = connect4_data.drop(columns=['outcome', 'player1_centercol', 'player1_bottomrow', 'player2_centercol', 'player2_bottomrow'])
outcome = connect4_data[['player1_centercol', 'player1_bottomrow', 'player2_centercol', 'player2_bottomrow', 'outcome']]

# Perform one-hot encoding on the feature columns
features_encoded = pd.get_dummies(features, prefix_sep='_', dtype=int)

# Combine the encoded features with the outcome column
df_encoded = pd.concat([features_encoded, outcome], axis=1)

In [106]:
df_encoded

,a1_b,a1_o,a1_x,a2_b,a2_o,a2_x,a3_b,a3_o,a3_x,a4_b,...,g5_o,g5_x,g6_b,g6_o,g6_x,player1_centercol,player1_bottomrow,player2_centercol,player2_bottomrow,outcome
0,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,3.0,2.0,3.0,0.0,win
1,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,3.0,2.0,3.0,1.0,win
2,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,3.0,2.0,3.0,1.0,win
3,1,0,0,1,0,0,1,0,0,1,...,0,0,1,0,0,3.0,2.0,3.0,1.0,win
4,0,1,0,1,0,0,1,0,0,1,...,0,0,1,0,0,3.0,2.0,3.0,1.0,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67549,0,0,1,0,0,1,0,1,0,0,...,0,0,1,0,0,0.0,1.0,0.0,2.0,loss
67550,0,0,1,1,0,0,1,0,0,1,...,0,0,1,0,0,0.0,2.0,0.0,2.0,loss
67551,0,0,1,0,0,1,0,1,0,1,...,0,0,1,0,0,0.0,2.0,0.0,1.0,loss
67552,0,0,1,0,0,1,1,0,0,1,...,0,0,1,0,0,0.0,1.0,0.0,2.0,loss


In [107]:
connect4_dict = df_encoded.to_dict(orient='records')

In [108]:
connect4_modified = connect4_dict

In [122]:
index_90 = int(len(connect4_modified) * 0.9)
X_train = [[1] + list(d.values())[:-1] for d in connect4_modified[:index_90]]
y_train = [d['outcome'] for d in connect4_modified[:index_90]]
X_test = [[1] + list(d.values())[:-1] for d in connect4_modified[index_90:]]
y_test = [d['outcome'] for d in connect4_modified[index_90:]]

In [123]:
X_train[0]

[1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 3.0,
 2.0,
 3.0,
 0.0]

In [131]:
mod = sklearn.linear_model.LogisticRegression(fit_intercept=False, class_weight = 'balanced', max_iter=1000)
mod.fit(X_train,y_train)

LogisticRegression(class_weight='balanced', fit_intercept=False, max_iter=1000)

In [132]:
predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test # Binary vector indicating which predictions were correct
sum(correct) / len(correct)

0.8098510882016037

In [133]:
TP = sum([(p=='win' and l=='win') for (p,l) in zip(predictions, y_test)])
FP = sum([(p=='win' and l=='loss') for (p,l) in zip(predictions, y_test)])
TN = sum([(p=='loss' and l=='loss') for (p,l) in zip(predictions, y_test)])
FN = sum([(p=='loss' and l=='win') for (p,l) in zip(predictions, y_test)])

print("TP = " + str(TP))
print("FP = " + str(FP))
print("TN = " + str(TN))
print("FN = " + str(FN))

TP = 3415
FP = 515
TN = 1534
FN = 647


In [137]:
TPR = TP / (TP + FN)
TNR = TN / (TN + FP)
print("TPR: " + str(TPR))
print("TNR: " + str(TNR))

BER = 1 - 1/2 * (TPR + TNR)
print("BER: " + str(BER))

TPR: 0.8407188577055638
TNR: 0.7486578818936066
BER: 0.20531163020041476


In [138]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)

precision, recall

(0.8689567430025446, 0.8407188577055638)